In [1]:
import os
import sys
from __future__ import print_function

DEPRECATED_HOME = '/root'

PROJECT_DIR = '~/shared/Documents/final_proj'
PROJECT_DIR = os.path.expanduser(PROJECT_DIR)

BASE_MODELS_DIR = os.path.join(PROJECT_DIR, 'models/trained_models')
ATTR_MODELS_DIR = os.path.join(BASE_MODELS_DIR, 'attribute_models')
BASE_FEATURES_DIR = os.path.join(PROJECT_DIR, 'extracted_features')
SVM_MODELS_DIR = os.path.join(PROJECT_DIR, 'models/svm_models')
SPLITS_DIR = os.path.join(PROJECT_DIR,'datasets/splits/annotator_consistency3')
STATS_MODELS_DIR = os.path.join(SVM_MODELS_DIR, 'stats')
LOGS_DIR = os.path.join(PROJECT_DIR, 'logs')

ARCH = 'mobileNet'
LAYER = 'fc7'

DOMAIN='domain'
RELATION='relation'
DATA_TYPE=DOMAIN

CONFIG = LAYER + '_' + DATA_TYPE + '_' + ARCH

DOMAIN_CLASSES, RELATION_CLASSES = 5, 16

if DATA_TYPE == DOMAIN:
    N_CLASSES = DOMAIN_CLASSES
    labels_path = os.path.join(SPLITS_DIR,'domain_single_body1_{}_5.txt')
else:
    N_CLASSES = RELATION_CLASSES
    labels_path = os.path.join(SPLITS_DIR,'single_body1_{}_16.txt')    

LABELS = [str(label) for label in range(N_CLASSES)]    

end2end_model = True

if end2end_model:
    FEATURES_DIR = os.path.join(BASE_FEATURES_DIR, 'end_to_end_features', CONFIG)    
else:    
    FEATURES_DIR = os.path.join(BASE_FEATURES_DIR, 'attribute_features', CONFIG)

stored_features_dir = os.path.join(FEATURES_DIR, 'all_splits_numpy_format')

N_STREAMS = 2


In [2]:
import keras

Using TensorFlow backend.


In [3]:
import numpy as np
from keras.preprocessing.image import load_img
from keras.utils import to_categorical

KERAS_TO_CAFFE_SPLITS = {'train': 'train', 'test': 'eval', 'val': 'test'}
FILE_FORMAT = '{}single_body'.format('domain_' if DATA_TYPE == DOMAIN else "") \
            + '{input_idx}_{split}_' + str(N_CLASSES) + '.txt'

STREAM_IDS = [str(idx) for idx in [1, 2]]

def process_line(line):
    image_path, label = line.strip().split(' ')
    image_path = image_path.replace(DEPRECATED_HOME, '~')

    if '~' in image_path:
        image_path = os.path.expanduser(image_path)

    return image_path, label

def get_file_name(split, input_idx, f_pattern=FILE_FORMAT):
    return f_pattern.format(split=KERAS_TO_CAFFE_SPLITS[split],
                            input_idx=input_idx)

def get_split_mapping(files_dir, splits, input_indices, f_pattern=FILE_FORMAT):
    mapping = {}
    for split in splits:
        for input_idx in input_indices:
            key = os.path.join(split, input_idx)
            value =  os.path.join(files_dir, get_file_name(split, input_idx, f_pattern))
            mapping[key] = value
    
    return mapping

def create_fake_directory(root_dir, split_mapping, labels, process_line_cb):
    if not os.path.exists(root_dir):
        os.mkdir(root_dir)
    
    for images_dir, images_file in split_mapping.items():
        abs_images_dir = os.path.join(root_dir, images_dir)
        
        for label in labels:
            label_dir = os.path.join(abs_images_dir, label)
            if not os.path.exists(label_dir):
                os.makedirs(label_dir)

        abs_images_file = os.path.join(abs_images_dir, images_file)
        with open(abs_images_file) as f:
            for fake_id, line in enumerate(f):
                image_path, label = process_line_cb(line)
                fake_name = '{}{}'.format(fake_id, os.path.splitext(image_path)[1])
                fake_image_path = os.path.join(abs_images_dir, label, fake_name)

                if not os.path.exists(fake_image_path):
                    os.symlink(image_path, fake_image_path)

In [4]:
import shutil

split_mapping = get_split_mapping(SPLITS_DIR, KERAS_TO_CAFFE_SPLITS.keys(), 
                                  STREAM_IDS)

FAKE_DIR = os.path.join(PROJECT_DIR,'datasets', 'fake_dir')

shutil.rmtree(FAKE_DIR)
create_fake_directory(FAKE_DIR, split_mapping, LABELS, process_line)

In [5]:
import threading

class threadsafe_iter(object):
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
            return next(self.it)


def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g

In [6]:
def relationToDomainLabel(rel_label):
    rel_label = int(rel_label)
    
    if rel_label in range(0, 4):
        return 0
    elif rel_label in range(4, 7):
        return 1
    elif rel_label in range(7, 8):
        return 2
    elif rel_label in range(8, 12):
        return 3
    elif rel_label in range(12, 16):
        return 4
    
    raise LookupError('Label out of range: {}. Valid range: [0,16).'.format(rel_label))

relationToDomainLabelVec = np.vectorize(relationToDomainLabel)    

In [7]:
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

IMAGE_SHAPE = (256, 256, 3)
#INPUT_SHAPE = (224, 224, 3)
INPUT_SHAPE = (227, 227, 3)

SHARED_SEED = 1
BATCH_SIZE = 32

def crop_transformation(preprocessing_function, center_crop_size):
    def center_crop(x):
        centerw, centerh = x.shape[0]//2, x.shape[1]//2
        halfw, halfh = center_crop_size[0]//2, center_crop_size[1]//2
        new_x = x[centerw-halfw:centerw+halfw,centerh-halfh:centerh+halfh, :]        
        return preprocessing_function(new_x)
    
    return center_crop

@threadsafe_generator
def multiple_stream_generator(generators, input_names): 
    assert len(generators) == len(input_names)
    
    while True:
        data_batch = [next(gen) for gen in generators]
        # multiple inputs
        X = {s_id : data_batch[idx][0] for idx, s_id in enumerate(input_names)}
        # just one output (all outputs are the same)
        y_rel = data_batch[0][1]
        # y_dom = to_categorical(relationToDomainLabelVec(np.argmax(y_rel, axis=1)), DOMAIN_CLASSES)       
        y = { 
           #  'output_domain' : y_dom
           # , 'output_relation' : y_rel
            'output_domain' : data_batch[1][1]
            }
        
        yield X, y
        
def create_dataset_generators(directory, classes, batch_size, seed=0, 
                              input_name_pattern='input_s{}',
                              preprocessing_function=None):
    generators = {}
    input_names = [input_name_pattern.format(s_id)  for s_id in STREAM_IDS]
    
    for split in ['train', 'test', 'val']:
        flow_gens = []

        for s_id in STREAM_IDS:
            if split == 'train':
                datagen = ImageDataGenerator(rescale=1./255,
                                             width_shift_range=0.2,
                                             height_shift_range=0.2,
                                             rotation_range=30,
                                             zoom_range=0.2,
                                             horizontal_flip=True,
                                             preprocessing_function=preprocessing_function)
            else:
                datagen = ImageDataGenerator(rescale=1./255,
                                             preprocessing_function=preprocessing_function)

            target_size=(INPUT_SHAPE[0], INPUT_SHAPE[1])
            images_dir = os.path.join(directory, split, s_id)
            flow_gen = datagen.flow_from_directory(
                                images_dir,
                                target_size=target_size,
                                batch_size=batch_size,
                                classes=classes,
                                seed=seed,
                                follow_links=True)
        
            flow_gens.append(flow_gen)
        
        generators[split] = multiple_stream_generator(flow_gens, input_names)

    return generators

In [8]:
from keras.applications import mobilenet
from keras.applications import imagenet_utils

generators = create_dataset_generators(FAKE_DIR, LABELS, BATCH_SIZE, SHARED_SEED, 
                                       preprocessing_function=imagenet_utils.preprocess_input)
#                                       preprocessing_function=mobilenet.preprocess_input)

Found 13729 images belonging to 5 classes.
Found 13729 images belonging to 5 classes.
Found 5106 images belonging to 5 classes.
Found 5106 images belonging to 5 classes.
Found 709 images belonging to 5 classes.
Found 709 images belonging to 5 classes.


In [9]:
#next(generators['val'])[1]['output_relation'][0].shape

In [10]:
N_TRAIN, N_TEST, N_VAL = 13729, 5106, 709

In [18]:
import keras

from keras.preprocessing import image
from keras.applications import mobilenet
from keras import regularizers
from keras.models import Model
from keras.models import load_model
from keras.layers import Dense, GlobalAveragePooling2D, Reshape, Dropout, Conv2D, Activation
from keras import backend as K
from keras.callbacks import ModelCheckpoint, TensorBoard

from keras_squeezenet import squeezenet

def create_model(stream_ids, classes, dropout=1e-3):

    # create the base pre-trained models
    arch = 'SqueezeNet'
    if arch == 'MobileNet':
        Net = mobilenet.MobileNet
    else:
        Net = squeezenet.SqueezeNet
    
    streams = {s_id : Net(weights='imagenet', 
                          include_top=False, 
                          pooling = 'avg',
#                          alpha=0.75,
                          input_shape=INPUT_SHAPE)
               for s_id in stream_ids}

    # fuse streams
    x = keras.layers.concatenate([stream.output for stream in streams.values()])
    
    # add top
    x = Reshape((1,1,-1), name='reshape_concat')(x)
    x_dropout = Dropout(dropout, name='dropout_concat')(x)
    
    outputs = []
    for data_type, n_classes in [
                                  ('domain', DOMAIN_CLASSES)
                                # , ('relation', RELATION_CLASSES)
                                ]:
        x = Conv2D(n_classes, (1, 1), padding='same', 
                   name='conv_preds_{}'.format(data_type))(x_dropout)
        
        x = Activation('softmax', name='act_softmax_{}'.format(data_type))(x)
        
        prediction = Reshape((n_classes,), 
                              name='output_{}'.format(data_type))(x)
        outputs.append(prediction)

    # rename layers to provide unique names
    for s_id, stream in streams.items():
        stream.layers[0].name = 'input'
        for idx, layer in enumerate(stream.layers):
            stream.layers[idx].name = layer.name + '_s' + s_id    
    
    # this is the model we will train
    model = Model(inputs=[streams[s_id].input for s_id in stream_ids], outputs=outputs)
    
    # regularization
    min_params_for_l2_reg = 100000000 # 50000
    l2_lambda = 0.01    
    for layer in model.layers:
        if type(layer) == Conv2D and layer.count_params() > min_params_for_l2_reg:
            layer.kernel_regularizer = regularizers.l2(l2_lambda)    
    
    return model

In [26]:
reuse_model = False
train_model = not reuse_model

checkpoint_path = os.path.join(BASE_MODELS_DIR, '{}_models'.format(ARCH), 
                               'weights.{epoch:02d}-{val_loss:.2f}.hdf5')
if reuse_model:
    last_epoch, val_loss = 10, 2.04
    model_file = checkpoint_path.format(epoch=last_epoch, val_loss=val_loss)

    if os.path.exists(model_file):
        model = load_model(model_file, custom_objects={
                           'relu6': mobilenet.relu6,
                           'DepthwiseConv2D': mobilenet.DepthwiseConv2D})
    else:
        create_model = True

if train_model:
    model = create_model(STREAM_IDS, N_CLASSES, dropout=0.1)

In [27]:
model.save(checkpoint_path.format(epoch=0, val_loss=0.0))
checkpoint_path = os.path.join(BASE_MODELS_DIR, '{}_models'.format(ARCH), 
                               'weights.{epoch:02d}-{val_loss:.2f}.hdf5')

model = keras.models.load_model(checkpoint_path.format(epoch=0, val_loss=0.0), 
                                custom_objects={
                           'relu6': mobilenet.relu6,
                           'DepthwiseConv2D': mobilenet.DepthwiseConv2D})

/src/keras/models.py:252: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [28]:
for idx, layer in enumerate(model.layers):
    layer.trainable = True
    print(idx, layer.name)
    
freeze = True
if freeze:
    n_blocks = 11 # set from 0 to 13
    freeze_idx = 164 - 6 * 2 * n_blocks
    
    freeze_idx = 126
    for layer in model.layers[:freeze_idx]:
        layer.trainable = not freeze

0 input_s2
1 input_s1
2 conv1_s2
3 conv1_s1
4 relu_conv1_s2
5 relu_conv1_s1
6 pool1_s2
7 pool1_s1
8 fire2/squeeze1x1_s2
9 fire2/squeeze1x1_s1
10 fire2/relu_squeeze1x1_s2
11 fire2/relu_squeeze1x1_s1
12 fire2/expand1x1_s2
13 fire2/expand3x3_s2
14 fire2/expand1x1_s1
15 fire2/expand3x3_s1
16 fire2/relu_expand1x1_s2
17 fire2/relu_expand3x3_s2
18 fire2/relu_expand1x1_s1
19 fire2/relu_expand3x3_s1
20 fire2/concat_s2
21 fire2/concat_s1
22 fire3/squeeze1x1_s2
23 fire3/squeeze1x1_s1
24 fire3/relu_squeeze1x1_s2
25 fire3/relu_squeeze1x1_s1
26 fire3/expand1x1_s2
27 fire3/expand3x3_s2
28 fire3/expand1x1_s1
29 fire3/expand3x3_s1
30 fire3/relu_expand1x1_s2
31 fire3/relu_expand3x3_s2
32 fire3/relu_expand1x1_s1
33 fire3/relu_expand3x3_s1
34 fire3/concat_s2
35 fire3/concat_s1
36 pool3_s2
37 pool3_s1
38 fire4/squeeze1x1_s2
39 fire4/squeeze1x1_s1
40 fire4/relu_squeeze1x1_s2
41 fire4/relu_squeeze1x1_s1
42 fire4/expand1x1_s2
43 fire4/expand3x3_s2
44 fire4/expand1x1_s1
45 fire4/expand3x3_s1
46 fire4/relu_expa

In [29]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_s2 (InputLayer)           (None, 227, 227, 3)  0                                            
__________________________________________________________________________________________________
input_s1 (InputLayer)           (None, 227, 227, 3)  0                                            
__________________________________________________________________________________________________
conv1_s2 (Conv2D)               (None, 113, 113, 64) 1792        input_s2[0][0]                   
__________________________________________________________________________________________________
conv1_s1 (Conv2D)               (None, 113, 113, 64) 1792        input_s1[0][0]                   
__________________________________________________________________________________________________
relu_conv1

In [30]:
callbacks = [ 
              ModelCheckpoint(checkpoint_path, monitor='val_loss', period=1)
            , TensorBoard(log_dir=LOGS_DIR, write_images=True, write_graph=True)
            ]

pretraining = True
pretraining_epochs = 10
if pretraining:
    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional Mobilenet layers
    # freeze(model, n_blocks='all')

    # compile the model (should be done *after* setting layers to non-trainable)
    optimizer = 'adam' # 'rmsprop'
    model.compile(optimizer=optimizer, loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # train the model on the new data for a few epochs
    model.fit_generator(
            generators['train'],
            steps_per_epoch=np.ceil(N_TRAIN / BATCH_SIZE),
            epochs=pretraining_epochs,
            validation_data=generators['val'],
            validation_steps=np.ceil(N_VAL / BATCH_SIZE),
            callbacks=callbacks,
            workers=4,
            max_queue_size=5)

    metrics = model.evaluate_generator(generators['test'], 
                                       steps=np.ceil(N_TEST / BATCH_SIZE),
                                       max_queue_size=5, workers=4)    
    print(metrics)
    
    last_epoch = pretraining_epochs

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.
post_training = True
post_training_epochs = 20
if post_training:
    # unfreeze the model
    # unfreeze(model, n_blocks='all')
    # freeze 3 blocks
    # freeze(model, n_blocks=3)

    # we need to recompile the model for these modifications to take effect
    # we use SGD with a low learning rate
    from keras.optimizers import SGD
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # we train our model again (this time fine-tuning the top 2 inception blocks
    # alongside the top Dense layers
    model.fit_generator(
            generators['train'],
            steps_per_epoch=np.ceil(N_TRAIN / BATCH_SIZE),
            epochs=last_epoch + post_training_epochs,
            validation_data=generators['val'],
            validation_steps=np.ceil(N_VAL / BATCH_SIZE),
            callbacks=callbacks, initial_epoch=last_epoch,
            workers=4, max_queue_size=5)

    metrics = model.evaluate_generator(generators['test'], 
                                       steps=np.ceil(N_TEST / BATCH_SIZE), 
                                       workers=4, max_queue_size=5)

    print(metrics)

Epoch 1/10
430/430 [==============================] - 254s 591ms/step - loss: 1.2309 - acc: 0.5748 - val_loss: 1.5724 - val_acc: 0.4598
Epoch 2/10
430/430 [==============================] - 227s 529ms/step - loss: 1.0096 - acc: 0.6389 - val_loss: 1.6461 - val_acc: 0.4429
Epoch 3/10
430/430 [==============================] - 233s 541ms/step - loss: 0.9473 - acc: 0.6643 - val_loss: 1.5981 - val_acc: 0.4104
Epoch 4/10
430/430 [==============================] - 221s 515ms/step - loss: 0.9208 - acc: 0.6771 - val_loss: 1.5599 - val_acc: 0.4133
Epoch 5/10
430/430 [==============================] - 232s 540ms/step - loss: 0.9101 - acc: 0.6767 - val_loss: 1.6857 - val_acc: 0.4260
Epoch 6/10
430/430 [==============================] - 224s 522ms/step - loss: 0.9046 - acc: 0.6746 - val_loss: 1.5558 - val_acc: 0.4245
Epoch 7/10
430/430 [==============================] - 234s 543ms/step - loss: 0.8875 - acc: 0.6872 - val_loss: 1.3776 - val_acc: 0.4217
Epoch 8/10
430/430 [============================

KeyboardInterrupt: 

In [31]:
metrics = model.evaluate_generator(generators['test'], 
                                   steps=np.ceil(N_TEST / BATCH_SIZE), 
                                   max_queue_size=5, workers=4)

print(metrics)

[1.1586970219040589, 0.5595377986682335]


In [ ]:
def check_tensorflow_gpu():
    import tensorflow as tf
    with tf.device('/gpu:0'):
        a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
        b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
        c = tf.matmul(a, b)

    with tf.Session() as sess:
        print (sess.run(c))

check_tensorflow_gpu()